In [1]:
# !md5sum /kaggle/input/qw14b-awq/transformers/default/1/model-00001-of-00002.safetensors
# should be 25596de367acaec20e616b7c87bd5529

In [2]:
import time

cutoff_time = int(time.time() + (8 * 60 + 50) * 60)

In [3]:
cutoff_time

1734005997

In [4]:
try:
    import vllm
except:
    !pip install torchvision==0.19.1
    !pip install vllm==0.6.3.post1
    !pip install optimum==1.23.3 autoawq==0.2.7.post2 auto-gptq==0.7.1  bitsandbytes==0.44.1 peft==0.12.0

WARNING 12-12 03:29:59 cuda.py:22] You are using a deprecated `pynvml` package. Please install `nvidia-ml-py` instead, and make sure to uninstall `pynvml`. When both of them are installed, `pynvml` will take precedence and cause errors. See https://pypi.org/project/pynvml for more information.


2024-12-12 03:30:14,097	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [5]:
import os, math, numpy as np
import sys
import os
from transformers import AutoTokenizer
import pandas as pd
from tqdm import tqdm
import re, gc
import torch
pd.set_option('display.max_rows', 300)

In [6]:
IS_SUBMISSION = bool(os.getenv("KAGGLE_IS_COMPETITION_RERUN"))

print('IS_SUBMISSION:', IS_SUBMISSION)

df_train = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv").fillna(-1).sample(10, random_state=42).reset_index(drop=True)
df_test = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv")
df_misconception_mapping = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")

if not IS_SUBMISSION:
    df_misconception_mapping = df_misconception_mapping[:100]

IS_SUBMISSION: False


# first retrieval

In [7]:
import pandas as pd
# from sentence_transformers import SentenceTransformer, util
if not IS_SUBMISSION:
    df_ret = df_test.copy()
else:
    df_ret = df_test.copy()

In [8]:
df_ret

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText
0,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets
1,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify
2,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct


In [9]:
TEMPLATE_INPUT_V3 = '{QUESTION}\nCorrect answer: {CORRECT_ANSWER}\nStudent wrong answer: {STUDENT_WRONG_ANSWER}'
def format_input_v3(row, wrong_choice):

    assert wrong_choice in "ABCD"
    # Extract values from the row
    question_text = row.get("QuestionText", "No question text provided")
    subject_name = row.get("SubjectName", "Unknown subject")
    construct_name = row.get("ConstructName", "Unknown construct")
    # Extract the correct and wrong answer text based on the choice
    correct_answer = row.get("CorrectAnswer", "Unknown")
    assert wrong_choice != correct_answer
    correct_answer_text = row.get(f"Answer{correct_answer}Text", "No correct answer text available")
    wrong_answer_text = row.get(f"Answer{wrong_choice}Text", "No wrong answer text available")

    # Construct the question format
    formatted_question = f"""Question: {question_text}
    
SubjectName: {subject_name}
ConstructName: {construct_name}"""

    # Return the extracted data
    ret = {
        "QUESTION": formatted_question,
        "CORRECT_ANSWER": correct_answer_text,
        "STUDENT_WRONG_ANSWER": wrong_answer_text,
        "MISCONCEPTION_ID": row.get('Misconception{wrong_choice}Id'),

        'ConstructName': row['ConstructName'],
        'SubjectName': row['SubjectName'],
        'QuestionText': row['QuestionText'],
        'CorrectAnswerText': correct_answer_text,
        'IncorrectAnswerText': wrong_answer_text,
    }
    ret["PROMPT"] = TEMPLATE_INPUT_V3.format(**ret)

    return ret


items = []
target_ids = []
for _, row in df_ret.iterrows():
    for choice in ['A', 'B', 'C', 'D']:
        if choice == row["CorrectAnswer"]:
            continue
        # if not IS_SUBMISSION and row[f'Misconception{choice}Id'] == -1:
        #     continue
            
        correct_col = f"Answer{row['CorrectAnswer']}Text"
        item = {'QuestionId_Answer': '{}_{}'.format(row['QuestionId'], choice)}
        item['Prompt'] = format_input_v3(row, choice)['PROMPT']

        item['ConstructName'] = row['ConstructName']
        item['SubjectName'] = row['SubjectName']
        item['QuestionText'] = row['QuestionText']
        correct_answer = row["CorrectAnswer"]
        item['CorrectAnswerText'] = row[f'Answer{correct_answer}Text']
        item['IncorrectAnswerText'] = row[f'Answer{choice}Text']
        
        items.append(item)
        target_ids.append(int(row.get(f'Misconception{choice}Id', -1)))
        
df_input = pd.DataFrame(items)

In [10]:
def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'<instruct>{task_description}\n<query>{query}'

def get_detailed_example(task_description: str, query: str, response: str) -> str:
    return f'<instruct>{task_description}\n<query>{query}\n<response>{response}'

def get_new_queries(queries, query_max_len, examples_prefix, tokenizer):
    inputs = tokenizer(
        queries,
        max_length=query_max_len - len(tokenizer('<s>', add_special_tokens=False)['input_ids']) - len(
            tokenizer('\n<response></s>', add_special_tokens=False)['input_ids']),
        return_token_type_ids=False,
        truncation=True,
        return_tensors=None,
        add_special_tokens=False
    )
    prefix_ids = tokenizer(examples_prefix, add_special_tokens=False)['input_ids']
    suffix_ids = tokenizer('\n<response>', add_special_tokens=False)['input_ids']
    new_max_length = (len(prefix_ids) + len(suffix_ids) + query_max_len + 8) // 8 * 8 + 8
    new_queries = tokenizer.batch_decode(inputs['input_ids'])
    for i in range(len(new_queries)):
        new_queries[i] = examples_prefix + new_queries[i] + '\n<response>'
    return new_max_length, new_queries
task =  "Given a math multiple-choice problem with a student's wrong answer, retrieve the math misconceptions"
queries = [
    get_detailed_instruct(task, q) for q in df_input['Prompt']
]
documents = df_misconception_mapping['MisconceptionName'].tolist()
query_max_len, doc_max_len = 1024, 256
LORA_PATH = '/kaggle/input/lora-14b-1126/transformers/default/1'
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/qw14b-awq/transformers/default/1')
examples_prefix = ''
new_query_max_len, new_queries = get_new_queries(queries, query_max_len, examples_prefix, tokenizer)


import json
with open('data.json', 'w') as f:
    data = {'texts': new_queries+ documents}
    f.write(json.dumps(data))

In [11]:
%%writefile run_embed.py
import argparse
import os
import json
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import peft

MAX_LENGTH = 2048


def last_token_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    left_padding = attention_mask[:, -1].sum() == attention_mask.shape[0]
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[
            torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths
        ]


def get_embeddings_in_batches(model, tokenizer, texts, max_length, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
        batch_texts = texts[i : i + batch_size]
        batch_dict = tokenizer(
            batch_texts,
            max_length=max_length,
            padding=True,
            truncation=True,
            return_tensors="pt",
        ).to("cuda")
        with torch.no_grad(), torch.amp.autocast("cuda"):
            outputs = model(**batch_dict)
            batch_embeddings = last_token_pool(
                outputs.last_hidden_state, batch_dict["attention_mask"]
            )
            batch_embeddings = F.normalize(batch_embeddings, p=2, dim=1).cpu()
        embeddings.append(batch_embeddings)
    return torch.cat(embeddings, dim=0)


def load_model_and_tokenizer(base_model_path, lora_path, load_in_4bit=True):
    model = AutoModel.from_pretrained(
        base_model_path,
        device_map=0,
        torch_dtype=torch.float16,
        load_in_4bit=load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        lora_path if lora_path else base_model_path
    )
    model.resize_token_embeddings(len(tokenizer))
    if lora_path:
        model = peft.PeftModel.from_pretrained(model, lora_path)
    return model, tokenizer


def main(args):
    output_file = args.input_text.replace(
        ".json", ".pt.fold.{}.{}.embed".format(*args.fold)
    )
    if os.path.exists(output_file):
        print(f"Output file {output_file} already exists. Skipping...")
        return
    model, tokenizer = load_model_and_tokenizer(
        args.base_model, args.lora_path, load_in_4bit=args.load_in_4bit
    )
    texts = json.load(open(args.input_text))["texts"][args.fold[0] :: args.fold[1]]
    embeddings = get_embeddings_in_batches(
        model,
        tokenizer,
        texts,
        max_length=MAX_LENGTH,
        batch_size=4,
    )
    text2embeds = {text: emb for text, emb in zip(texts, embeddings)}
    torch.save(text2embeds, output_file)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--base_model",
        type=str,
        default="Qwen/Qwen2.5-7B",
        help="Path to the base model",
    )
    parser.add_argument(
        "--lora_path",
        type=str,
        default=None,
        help="Path to the LoRA model",
    )
    parser.add_argument(
        "--input_text",
        type=str,
        default=".cache/data.json",
    )
    parser.add_argument(
        "--load_in_4bit",
        action="store_true",
        help="Load model in 4-bit mode",
    )
    parser.add_argument("--fold", nargs=2, type=int, default=[0, 1])
    args = parser.parse_args()
    if not os.path.exists(args.lora_path):
        args.lora_path = None
    main(args)


Writing run_embed.py


In [12]:
!sleep 1 & sleep 3

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
# %%writefile run.sh
lora_path = '/kaggle/input/2211-lora-14b/transformers/default/1'
cmd = f"(CUDA_VISIBLE_DEVICES=0 python run_embed.py --base_model /kaggle/input/qw14b-awq/transformers/default/1 --lora_path {lora_path} --input_text data.json --fold 0 2) & (CUDA_VISIBLE_DEVICES=1 python run_embed.py --base_model /kaggle/input/qw14b-awq/transformers/default/1 --lora_path {lora_path} --input_text data.json --fold 1 2)"
import os
os.system(cmd)

Embedding:  71%|███████▏  | 10/14 [00:16<00:02,  1.49it/s]

0

In [14]:
from glob import glob
import time
text_to_embed = {}
files = glob('*.pt*')
while len(files) != 2:
    time.sleep(1)
    files = glob('*.pt*')

time.sleep(3)    
for path in files:
    print(path)
    text_to_embed.update(torch.load(path))

Embedding: 100%|██████████| 14/14 [00:18<00:00,  1.31s/it]


data.pt.fold.1.2.embed
data.pt.fold.0.2.embed


/tmp/ipykernel_40/3571780413.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  text_to_embed.update(torch.load(path))


In [15]:
len(text_to_embed)

109

In [16]:
query_embeddings = torch.stack([text_to_embed[t] for t in new_queries])
doc_embeddings = torch.stack([text_to_embed[t] for t in documents])
query_embeddings.shape, doc_embeddings.shape

(torch.Size([9, 5120]), torch.Size([100, 5120]))

In [17]:
eedi_11scores = query_embeddings @ doc_embeddings.T  # Shape: (M, N)
sorted_indices = torch.argsort(eedi_11scores,1, descending=True)[:,:35].tolist()

In [18]:
df_input["MisconceptionId"] = [" ".join([str(x) for x in row]) for row in sorted_indices]
df_input[["QuestionId_Answer", "MisconceptionId"]].to_csv("submission.csv", index=False)

In [19]:
pd.read_csv('submission.csv')

,QuestionId_Answer,MisconceptionId
0,1869_B,15 77 23 96 25 74 90 52 55 95 30 94 78 57 11 1...
1,1869_C,15 77 74 23 90 96 71 37 52 25 35 13 55 30 65 8...
2,1869_D,15 77 25 96 90 74 95 14 4 65 44 68 52 30 71 64...
3,1870_A,91 59 82 80 4 79 32 27 55 3 74 29 34 44 68 94 ...
4,1870_B,91 80 59 82 79 32 3 27 29 55 4 74 44 46 68 34 ...
5,1870_C,82 91 80 59 55 79 74 3 34 4 32 68 27 29 46 52 ...
6,1871_A,14 62 23 24 30 94 63 77 38 74 25 93 32 65 52 6...
7,1871_C,14 62 24 23 30 94 63 32 77 93 25 74 6 38 65 52...
8,1871_D,23 14 63 94 24 74 77 89 65 62 32 30 42 18 29 2...


In [20]:
stage2_data = []
for i, row in df_input.iterrows():
    qa_id = row['QuestionId_Answer']
    top_mis_ids = sorted_indices[i]
        
    stage2_data.append({
        'qa_id': qa_id,
        'ConstructName': row['ConstructName'],
        'SubjectName': row['SubjectName'],
        'QuestionText': row['QuestionText'],
        'CorrectAnswerText': row[f'CorrectAnswerText'],
        'IncorrectAnswerText': row[f'IncorrectAnswerText'],
        # 'gt_mis_id': int(gt_mis_id),
        'top_mis_id': top_mis_ids,
    })

In [21]:
stage2_data[0]

{'qa_id': '1869_B',
 'ConstructName': 'Use the order of operations to carry out calculations involving powers',
 'SubjectName': 'BIDMAS',
 'QuestionText': '\\[\n3 \\times 2+4-5\n\\]\nWhere do the brackets need to go to make the answer equal \\( 13 \\) ?',
 'CorrectAnswerText': '\\( 3 \\times(2+4)-5 \\)',
 'IncorrectAnswerText': '\\( 3 \\times 2+(4-5) \\)',
 'top_mis_id': [15,
  77,
  23,
  96,
  25,
  74,
  90,
  52,
  55,
  95,
  30,
  94,
  78,
  57,
  11,
  1,
  68,
  37,
  34,
  82,
  60,
  14,
  64,
  36,
  4,
  44,
  38,
  32,
  31,
  81,
  19,
  35,
  58,
  24,
  71]}

In [22]:
with open('stage2_data.json', 'w') as file:
    json.dump(stage2_data, file)

# My Q14B Retrieval

In [23]:
QUERY_TEMPLATE = """<subject>
{SubjectName}
</subject>

<construct>
{ConstructName}
</construct>

<question>
{QuestionText}
</question>

<correct_answer>
{CorrectAnswerText}
</correct_answer>

<incorrect_answer>
{IncorrectAnswerText}
</incorrect_answer>"""

TASK_DESC = 'Given a math question with correct answer and a misconcepted incorrect answer, retrieve the most accurate misconception for the incorrect answer.'

INST_TEMPLATE = 'Instruct: {instruct}\n\nQuery: {query}'

In [24]:
import pandas as pd
import os

full_df = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv")
mis_map = pd.read_csv('/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv')
IS_SUBMISSION = bool(os.getenv("KAGGLE_IS_COMPETITION_RERUN"))

if not IS_SUBMISSION:
    mis_map = mis_map[:100]

rows = []
for idx, row in full_df.iterrows():
    for option in ["A", "B", "C", "D"]:
        if option == row.CorrectAnswer:
            continue
            
        correct_answer = row[f"Answer{row.CorrectAnswer}Text"]
        incorrect_answer = row[f"Answer{option}Text"]

        query_text = QUERY_TEMPLATE.format(**{
            'SubjectName': row['SubjectName'],
            'ConstructName': row['ConstructName'],
            'QuestionText': row['QuestionText'],
            'CorrectAnswerText': correct_answer,
            'IncorrectAnswerText': incorrect_answer,
        })
        full_instruct = row['query_text'] = INST_TEMPLATE.format(
            instruct=TASK_DESC,
            query=query_text,
        )

        rows.append({
            "qa_id": f"{row.QuestionId}_{option}",
            "query_text": query_text, 
            "full_instruct": full_instruct,
            "ConstructName": row.ConstructName,
            "SubjectName": row.SubjectName,
            "QuestionText": row.QuestionText,
            "correct_answer": correct_answer,
            "incorrect_answer": incorrect_answer,
        })
        
df = pd.DataFrame(rows)
df.head()

,qa_id,query_text,full_instruct,ConstructName,SubjectName,QuestionText,correct_answer,incorrect_answer
0,1869_B,<subject>\nBIDMAS\n</subject>\n\n<construct>\n...,Instruct: Given a math question with correct a...,Use the order of operations to carry out calcu...,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \)
1,1869_C,<subject>\nBIDMAS\n</subject>\n\n<construct>\n...,Instruct: Given a math question with correct a...,Use the order of operations to carry out calcu...,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times(2+4-5) \)
2,1869_D,<subject>\nBIDMAS\n</subject>\n\n<construct>\n...,Instruct: Given a math question with correct a...,Use the order of operations to carry out calcu...,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),Does not need brackets
3,1870_A,<subject>\nSimplifying Algebraic Fractions\n</...,Instruct: Given a math question with correct a...,Simplify an algebraic fraction by factorising ...,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",Does not simplify,\( m+1 \)
4,1870_B,<subject>\nSimplifying Algebraic Fractions\n</...,Instruct: Given a math question with correct a...,Simplify an algebraic fraction by factorising ...,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",Does not simplify,\( m+2 \)


In [25]:
question_texts = df['query_text'].values.tolist()
mis_texts = mis_map['MisconceptionName'].values.tolist()

print(f"Num Question: {len(question_texts)}")
print(f"Num Misconceptions: {len(mis_texts)}")

all_text_to_embed = question_texts + mis_texts

with open('stage1_input_texts.json', 'w') as file:
    json.dump(all_text_to_embed, file)

Num Question: 9
Num Misconceptions: 100


In [26]:
%%writefile run_stage1.py

import os
import sys
import json
from tqdm import tqdm, trange
import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
from peft import PeftModel
from transformers import AutoTokenizer, AutoModel

MODEL_PATH = sys.argv[1]
LORA_PATH = sys.argv[2]
SAVE_PATH = sys.argv[3]
LEFT_PADDING = True

with open('stage1_input_texts.json', 'r') as file:
    all_text_to_embed = json.load(file)

def last_token_pool(last_hidden_states, attention_mask, left_padding=None):
    if left_padding is None:
        left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
        
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]
    
def inference(sentences, model, tokenizer, device):
    batch_size = 16
    max_length = 2048
    
    length_sorted_idx = np.argsort([-len(sen) for sen in sentences])
    sentences_sorted = [sentences[idx] for idx in length_sorted_idx]
    
    # Get embeddings in sorted order
    all_embeddings = []
    for start_index in tqdm(range(0, len(sentences), batch_size), desc="Batches"):
        sentences_batch = sentences_sorted[start_index: start_index + batch_size]
        encoded_batch = tokenizer(
            sentences_batch,
            max_length=max_length,
            padding=True,
            truncation=True,
            return_tensors="pt",
        )
        encoded_batch = encoded_batch.to(device)
        with torch.no_grad():
            outputs = model(**encoded_batch)
            embeddings = last_token_pool(
                outputs.last_hidden_state, 
                encoded_batch['attention_mask'], 
                left_padding=LEFT_PADDING,
            )
            embeddings = torch.nn.functional.normalize(embeddings, dim=-1)
            embeddings = embeddings.detach().cpu().numpy()
        all_embeddings.append(embeddings)

    all_embeddings = np.concatenate(all_embeddings, axis=0)
    rearranged_embeddings = all_embeddings[np.argsort(length_sorted_idx)]
    
    return rearranged_embeddings

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
if LEFT_PADDING:
    tokenizer.padding_side = 'left'
else:
    tokenizer.padding_side = 'right'

model = AutoModel.from_pretrained(
    MODEL_PATH, 
    torch_dtype=torch.float16,
    device_map='cuda:0',
    # attn_implementation='flash_attention_2',
)

model = PeftModel.from_pretrained(model, LORA_PATH)

all_embeddings = inference(
    all_text_to_embed, 
    model, tokenizer, 'cuda:0'
)

np.save(SAVE_PATH, all_embeddings)

Writing run_stage1.py


In [27]:
! CUDA_VISIBLE_DEVICES=0 python run_stage1.py \
    /kaggle/input/qwen2.5/transformers/qwen2.5-14b-instruct-awq/1/Qwen2.5-14B-Instruct-AWQ \
    /kaggle/input/eedi-kfold-embedder/stage1_sep_formatted/q14b_ep5_u2b2g8_fold_1 \
    q14_embedding_f1.npy & \
CUDA_VISIBLE_DEVICES=1 python run_stage1.py \
    /kaggle/input/qwen2.5/transformers/qwen2.5-14b-instruct-awq/1/Qwen2.5-14B-Instruct-AWQ \
    /kaggle/input/eedi-kfold-embedder/stage1_sep_formatted/q14b_ep5_u2b2g8_fold_2 \
    q14_embedding_f2.npy 1 & \
wait

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Batches: 100%|████████████████████████████████████| 7/7 [00:17<00:00,  2.44s/it]


In [28]:
# ! CUDA_VISIBLE_DEVICES=0 python run_stage1.py \
#     /kaggle/input/qwen2.5/transformers/qwen2.5-14b-instruct-awq/1/Qwen2.5-14B-Instruct-AWQ \
#     /kaggle/input/eedi-kfold-embedder/stage1_sep_formatted/q14b_ep5_u2b2g8_fold_3 \
#     q14_embedding_f3.npy & \
# CUDA_VISIBLE_DEVICES=1 python run_stage1.py \
#     /kaggle/input/qwen2.5/transformers/qwen2.5-14b-instruct-awq/1/Qwen2.5-14B-Instruct-AWQ \
#     /kaggle/input/eedi-kfold-embedder/stage1_sep_formatted/q14b_ep5_u2b2g8_fold_4 \
#     q14_embedding_f4.npy 1 & \
# wait

In [29]:
# ! CUDA_VISIBLE_DEVICES=0 python run_stage1.py \
#     /kaggle/input/qwen2.5/transformers/qwen2.5-14b-instruct-awq/1/Qwen2.5-14B-Instruct-AWQ \
#     /kaggle/input/eedi-kfold-embedder/stage1_sep_formatted/qwen_14b_fold_1_ep5_bsz4_fix \
#     q14_embedding_f1.npy & \
# CUDA_VISIBLE_DEVICES=1 python run_stage1.py \
#     /kaggle/input/qwen2.5/transformers/qwen2.5-14b-instruct-awq/1/Qwen2.5-14B-Instruct-AWQ \
#     /kaggle/input/eedi-kfold-embedder/stage1_sep_formatted/qwen_14b_fold_2_ep5_bsz4_fix \
#     q14_embedding_f2.npy 1 & \
# wait

In [30]:
! CUDA_VISIBLE_DEVICES=0 python run_stage1.py \
    /kaggle/input/qwen2.5/transformers/qwen2.5-14b-instruct-awq/1/Qwen2.5-14B-Instruct-AWQ \
    /kaggle/input/eedi-kfold-embedder/stage1_sep_formatted/qwen_14b_fold_3_ep5_bsz4_fix \
    q14_embedding_f3.npy & \
CUDA_VISIBLE_DEVICES=1 python run_stage1.py \
    /kaggle/input/qwen2.5/transformers/qwen2.5-14b-instruct-awq/1/Qwen2.5-14B-Instruct-AWQ \
    /kaggle/input/eedi-kfold-embedder/stage1_sep_formatted/qwen_14b_fold_4_ep5_bsz4_fix \
    q14_embedding_f4.npy 1 & \
wait

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Batches: 100%|████████████████████████████████████| 7/7 [00:17<00:00,  2.46s/it]


In [31]:
all_similarities = []
for i_fold in [1,2,3,4]:
    fold_embedding = np.load(f'q14_embedding_f{i_fold}.npy')
    q_embedding = fold_embedding[:len(question_texts)]
    m_embedding = fold_embedding[len(question_texts):]

    sim = q_embedding @ m_embedding.T
    all_similarities.append(sim)

In [32]:
mean_scores = np.mean(all_similarities + [eedi_11scores, eedi_11scores], axis=0)

In [33]:
# mean_scores = torch.tensor(eedi_11scores)
mean_scores = torch.tensor(mean_scores)

In [34]:
sorted_indices = torch.argsort(mean_scores, 1, descending=True)[:,:35].tolist()

stage2_data = []
for i, row in df_input.iterrows():
    qa_id = row['QuestionId_Answer']
    top_mis_ids = sorted_indices[i]
        
    stage2_data.append({
        'qa_id': qa_id,
        'ConstructName': row['ConstructName'],
        'SubjectName': row['SubjectName'],
        'QuestionText': row['QuestionText'],
        'CorrectAnswerText': row[f'CorrectAnswerText'],
        'IncorrectAnswerText': row[f'IncorrectAnswerText'],
        'top_mis_id': top_mis_ids,
    })

In [35]:
with open('stage2_data.json', 'w') as file:
    json.dump(stage2_data, file)

# VLLM

In [36]:
%%writefile run_vllm.py

import os
import sys
# os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"
import vllm
from vllm.lora.request import LoRARequest
import random
import numpy as np
import pandas as pd
from transformers import PreTrainedTokenizer, AutoTokenizer
from typing import List
import torch
import re
import json
import time

cutoff_time = int(sys.argv[1])

LORA_PATHS = [
    "/kaggle/input/eedi_qwen_reranker/transformers/eedi_stage2_q32b_ep5/1",
    "/kaggle/input/eedi_qwen_reranker/transformers/eedi_stage2_q32b_ep5_fold0/1",
    "/kaggle/input/eedi_qwen_reranker/transformers/eedi_stage2_q32b_ep5_fold1/1",
    "/kaggle/input/eedi_qwen_reranker/transformers/eedi_stage2_q32b_ep5_fold2/1",
    "/kaggle/input/eedi_qwen_reranker/transformers/eedi_stage2_q32b_ep5_fold3/1",
    "/kaggle/input/eedi_qwen_reranker/transformers/eedi_stage2_q32b_syn_ep2/1",
]
model_path = "/kaggle/input/qwen2.5/transformers/qwen2.5-32b-instruct-awq/1/Qwen2.5-32B-Instruct-AWQ"
tokenizer = AutoTokenizer.from_pretrained(model_path)

mis_map = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")
mis_list = mis_map["MisconceptionName"].values

with open('stage2_data.json', 'r') as f:
    stage2_data = json.load(f)

def preprocess_text(x):
    x = re.sub("http\w+", '',x)   # Delete URL
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x

PROMPT = """Task:
As a Mathematics teacher, your goal is to analyze a student's incorrect answer of a mathematics question, identify their fundamental conceptual misunderstanding, and select the single most appropriate misconception number from the given misconception options.

Here is a mathematics question about {ConstructName}({SubjectName}).
Question:
{Question}

Correct Answer:
{CorrectAnswer}

Incorrect Answer:
{IncorrectAnswer}

Carefully analyze the incorrect answer and select a single most appropriate misconception number from the given misconceptions.

Here are the retrieved misconceptions:
{Retrival}

Only output the code of the selected misconception.
Don't output any other words.
"""

mis_codes = [
    '1', '2', '3', '4', '5', '6', '7', '8', '9',
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I',
    'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
    'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
]


def apply_template(row, tokenizer):
    messages = [
        {
            "role": "user", 
            "content": preprocess_text(
                PROMPT.format(
                    ConstructName=row["ConstructName"],
                    SubjectName=row["SubjectName"],
                    Question=row["QuestionText"],
                    IncorrectAnswer=row[f"IncorrectAnswerText"],
                    CorrectAnswer=row[f"CorrectAnswerText"],
                    Retrival=row[f"retrieval"]
                )
            )
        }
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return text

def make_cand_txt(cond_mids, mis_list):
    candidate_mis_txts = [mis_list[x] for x in cond_mids]
    return '\n'.join([f'{mc}. {mt}' for mc, mt in zip(mis_codes, candidate_mis_txts)])

import random
from typing import List, Dict, Tuple

def local_shuffle_with_mapping(lst: List, max_distance: int = 5) -> Tuple[List, Dict[int, int]]:
    """
    對列表進行局部打亂，並返回新位置到原始位置的映射關係
    通過交換操作確保所有元素的移動距離都不超過指定的最大值
    
    參數:
    lst: 要打亂的列表
    max_distance: 元素最大移動距離（預設為5）
    
    返回:
    tuple: (打亂後的列表, {新位置: 原始位置} 的字典)
    """
    result = lst.copy()
    n = len(result)
    
    # 追蹤每個位置的元素的原始位置
    curr_to_orig = list(range(n))  # curr_to_orig[i] 表示當前在位置i的元素原本在哪個位置
    
    # 隨機選擇要處理的位置順序
    positions = list(range(n))
    random.shuffle(positions)
    
    for pos in positions:
        # 找出當前位置元素的原始位置
        orig_pos = curr_to_orig[pos]
        
        # 計算可以交換的範圍
        # 只考慮向前或向後max_distance以內的位置
        start = max(0, pos - max_distance)
        end = min(n - 1, pos + max_distance)
        valid_positions = []
        
        # 檢查每個可能的交換位置
        for new_pos in range(start, end + 1):
            if new_pos != pos:
                # 檢查如果與new_pos交換，兩個位置的元素是否都滿足最大距離限制
                other_orig_pos = curr_to_orig[new_pos]
                
                # 檢查兩個元素交換後是否都滿足距離限制
                if (abs(new_pos - orig_pos) <= max_distance and  # 當前元素移動到new_pos的距離
                    abs(pos - other_orig_pos) <= max_distance):  # new_pos位置的元素移動到pos的距離
                    valid_positions.append(new_pos)
        
        # 如果有合法的交換位置，隨機選一個進行交換
        if valid_positions:
            swap_pos = random.choice(valid_positions)
            
            # 交換元素
            result[pos], result[swap_pos] = result[swap_pos], result[pos]
            # 更新映射
            curr_to_orig[pos], curr_to_orig[swap_pos] = curr_to_orig[swap_pos], curr_to_orig[pos]
    
    # 創建最終的映射字典
    position_mapping = {i: curr_to_orig[i] for i in range(n)}
    
    return result, position_mapping

for d in stage2_data:
    top_mis_id = d['top_mis_id'][:35]

    s_mid_ids, new_to_old_pos_map = local_shuffle_with_mapping(
        top_mis_id,
        max_distance=5
    )
    retrieval_txt = make_cand_txt(s_mid_ids, mis_list)

    in_text = apply_template({
        **d,
        "retrieval": retrieval_txt
    }, tokenizer)
    d['in_text'] = in_text
    
print(stage2_data[0]['in_text'])

llm = vllm.LLM(
    model_path,
    quantization="awq",
    tensor_parallel_size=2,
    gpu_memory_utilization=0.90, 
    dtype="half", 
    enforce_eager=True,
    max_model_len=4096,
    # disable_log_stats=True,

    enable_lora=True,
    max_lora_rank=64,
    max_logprobs=40,
)


all_lora_preds = []
for i_lora, lora_path in enumerate(LORA_PATHS):
    remain_time = cutoff_time - time.time()
    print(f'Remain time: {remain_time}')
    if remain_time < 50 * 60:
        print('Remain time < 50 min, break')
        break
    
    lora_request = LoRARequest(f'lora_{i_lora}', i_lora+1, lora_path=lora_path)
    print(f"Using {i_lora+1}-th LoRA from {lora_path}")

    # shuffle top_mid_id
    for d in stage2_data:
        top_mis_id = d['top_mis_id'][:35]
    
        s_mid_ids, new_to_old_pos_map = local_shuffle_with_mapping(
            top_mis_id,
            max_distance=5
        )
        retrieval_txt = make_cand_txt(s_mid_ids, mis_list)
    
        in_text = apply_template({
            **d,
            "retrieval": retrieval_txt
        }, tokenizer)
        d['in_text'] = in_text
        d['new_to_old_pos_map'] = new_to_old_pos_map

    sampling_params = vllm.SamplingParams(
        n=1,
        temperature=0.0,
        max_tokens=1,
        logprobs=40,
    )
    
    llm_responses = llm.generate(
        [d['in_text'] for d in stage2_data],
        sampling_params,
        use_tqdm=True,
        lora_request=lora_request,
    )

    print('-----------MODEL OUTPUT EXAMPLE--------------')
    print(llm_responses[0].outputs[0].text)
    print('---------------------------------------------')

    all_preds = []
    for d, respones in zip(stage2_data, llm_responses):
        new_to_old_pos_map = d['new_to_old_pos_map']
        probs = [0.0] * len(mis_codes)
        for token_id, logprob in respones.outputs[0].logprobs[0].items():
            decoded_token = logprob.decoded_token
            lp = logprob.logprob
            if decoded_token in mis_codes:
                pred_index = mis_codes.index(decoded_token)
                pred_index = new_to_old_pos_map[pred_index]
                probs[pred_index] = np.exp(lp)
        all_preds.append(probs)
    all_preds = np.array(all_preds)

    all_lora_preds.append(all_preds)

mean_preds = np.mean(all_lora_preds, axis=0)

results = []
result_probs = []
all_top_mis_ids = [x['top_mis_id'] for x in stage2_data]
for top_mis_ids, pred_probs in zip(all_top_mis_ids, mean_preds):
    sorted_indices = np.argsort(pred_probs)[::-1]
    pred_mis_ids = [top_mis_ids[si] for si in sorted_indices if si < len(top_mis_ids)]
    pred_probs = [pred_probs[si] for si in sorted_indices if si < len(top_mis_ids)]
    assert len(pred_mis_ids) == len(pred_probs)
    results.append(pred_mis_ids)
    result_probs.append(pred_probs)
    
# all_pred_ranks = []
# for respones in llm_responses:
#     pred_ranks = []
#     for token_id, logprob in respones.outputs[0].logprobs[0].items():
#         decoded_token = logprob.decoded_token
#         if decoded_token in mis_codes:
#             pred_ranks.append(mis_codes.index(decoded_token))
#     all_pred_ranks.append(pred_ranks)
    
# results = []
# all_top_mis_ids = [x['top_mis_id'] for x in stage2_data]
# for top_mis_ids, pred_ranks in zip(all_top_mis_ids, all_pred_ranks):
#     pred_mis_ids = [top_mis_ids[pr] for pr in pred_ranks if pr < len(top_mis_ids)]
#     r = pred_mis_ids + [x for x in top_mis_ids if x not in pred_mis_ids]
#     results.append(r[:25])

result_txts = [' '.join([str(x) for x in mids]) for mids in results]

submission = pd.read_csv('/kaggle/input/eedi-mining-misconceptions-in-mathematics/sample_submission.csv')
submission["MisconceptionId"] = result_txts

submission.to_csv("submission.csv", index=False)
submission.to_csv("submission_vllm.csv", index=False)
print('Save result to submission.csv and submission_vllm.csv')

with open('result_probs.json', 'w') as file:
    json.dump(result_probs, file)
print('Save result_probs to result_probs.json')

Writing run_vllm.py


In [37]:
!python run_vllm.py $cutoff_time

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


WARNING 12-12 03:36:49 cuda.py:22] You are using a deprecated `pynvml` package. Please install `nvidia-ml-py` instead, and make sure to uninstall `pynvml`. When both of them are installed, `pynvml` will take precedence and cause errors. See https://pypi.org/project/pynvml for more information.
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Task:
As a Mathematics teacher, your goal is to analyze a student's incorrect answer of a mathematics question, identify their fundamental conceptual misunderstanding, and select the single most appropriate misconception number from the given misconception options.

Here is a mathematics question about Use the order of operations to carry out calculations involving powers(BIDMAS).
Question:
\[
3 \times 2+4-5
\]
Where do the brackets need to go to make the answer equal 13 ?

Correct Answer:
 3 \times(2+4)-5 

Incorrect Answer:
 3 \times 2+(4-5) 

Carefully analyze the incorrect answer

In [38]:
submission = pd.read_csv("submission_vllm.csv")
submission

,QuestionId_Answer,MisconceptionId
0,1869_B,55 15 77 24 34 52 74 23 64 82 60 96 84 13 90 1...
1,1869_C,15 52 77 82 25 34 74 73 31 68 23 29 46 96 32 6...
2,1869_D,15 77 68 55 24 74 31 64 14 95 46 60 96 13 25 4...
3,1870_A,59 80 91 79 32 3 29 46 94 25 44 6 27 4 55 49 6...
4,1870_B,59 80 79 58 55 46 82 44 32 3 6 64 91 74 29 73 ...
5,1870_C,59 80 46 3 79 49 82 64 44 32 91 74 34 4 89 27 ...
6,1871_A,30 99 93 71 46 65 64 41 86 23 34 16 98 38 2 79...
7,1871_C,30 65 79 38 94 78 64 71 41 98 13 49 14 93 89 7...
8,1871_D,30 89 84 49 23 56 38 93 52 46 94 64 27 99 77 8...


In [39]:
with open('result_probs.json') as file:
    result_probs = json.load(file)

result_probs[0]

[0.47902074456012,
 0.09341742497260075,
 0.0849409653579438,
 0.019503722015854753,
 0.019272562565414406,
 0.018412652093828436,
 0.015172199748685186,
 0.01448667154444854,
 0.014297883007457502,
 0.013540329067711167,
 0.013214858451031397,
 0.012986003898313775,
 0.011284559427117221,
 0.011016912369124625,
 0.011007700379314844,
 0.010772409173457906,
 0.010356878825827625,
 0.010268283527928775,
 0.010010688510497355,
 0.009678680140212921,
 0.00961114464388261,
 0.009220271514517467,
 0.00921780023514358,
 0.00919266449175064,
 0.009178374196190648,
 0.008064827325946674,
 0.007851268775416583,
 0.0076363869628934935,
 0.007547552857929776,
 0.006793104372738863,
 0.006034493123839287,
 0.00569695633673428,
 0.00531662286983058,
 0.004601367576941298,
 0.004168739390781904]

In [40]:
mis_table = pd.read_csv('/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv')
df_train = pd.read_csv('/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv')

mis_cols = ['MisconceptionAId', 'MisconceptionBId', 'MisconceptionCId', 'MisconceptionDId']
all_mis = df_train[mis_cols].values.flatten()
all_mis = [int(mid) for mid in all_mis if pd.notna(mid)]
print(len(all_mis), len(set(all_mis)))

novel_mis = [mis_id for mis_id in range(len(mis_table)) if mis_id not in all_mis]
len(novel_mis)

4370 1604


983

In [41]:
novel_mis_set = set(novel_mis)

In [42]:
all_preds = [
    [int(m) for m in p.split(' ')] for p in submission['MisconceptionId']
]
all_pred_probs = [ps for ps in result_probs]

all_mid_probs = []

# all_first = set([int(p[0]) for p in all_preds])
# novel_mis = [m for m in novel_mis if not m in all_first]
# len(novel_mis)

In [43]:
TARGET_NOVEL_FIRST_RATE = 0.75
novel_bonus = 1.0

while True:

    mod_pred_probs = []
    for i_question, (preds, probs) in enumerate(zip(all_preds, all_pred_probs)):
        pps = []
        for i_mis, (pred_id, prob) in enumerate(zip(preds, probs)):
            if pred_id in novel_mis_set:
                pps.append([pred_id, prob * novel_bonus])
            else:
                pps.append([pred_id, prob])
        mod_pred_probs.append(pps)

    mod_pred_probs = [
        sorted(pps, key=lambda x: x[1], reverse=True) for pps in mod_pred_probs
    ]

    first_novel_rate = np.mean([pps[0][0] in novel_mis_set for pps in mod_pred_probs])

    print(f'Bonus ratio: {round(novel_bonus, 2)}, first rate {first_novel_rate}')

    if first_novel_rate > TARGET_NOVEL_FIRST_RATE:
        print('Done')
        break
    elif novel_bonus > 100.0:
        print('Extreme high bonus (>100), cant find enough novel')
        break
    else:
        novel_bonus += 0.02

Bonus ratio: 1.0, first rate 0.6666666666666666
Bonus ratio: 1.02, first rate 0.6666666666666666
Bonus ratio: 1.04, first rate 0.6666666666666666
Bonus ratio: 1.06, first rate 0.6666666666666666
Bonus ratio: 1.08, first rate 0.6666666666666666
Bonus ratio: 1.1, first rate 0.6666666666666666
Bonus ratio: 1.12, first rate 0.6666666666666666
Bonus ratio: 1.14, first rate 0.6666666666666666
Bonus ratio: 1.16, first rate 0.6666666666666666
Bonus ratio: 1.18, first rate 0.6666666666666666
Bonus ratio: 1.2, first rate 0.6666666666666666
Bonus ratio: 1.22, first rate 0.6666666666666666
Bonus ratio: 1.24, first rate 0.6666666666666666
Bonus ratio: 1.26, first rate 0.6666666666666666
Bonus ratio: 1.28, first rate 0.6666666666666666
Bonus ratio: 1.3, first rate 0.6666666666666666
Bonus ratio: 1.32, first rate 0.6666666666666666
Bonus ratio: 1.34, first rate 0.6666666666666666
Bonus ratio: 1.36, first rate 0.6666666666666666
Bonus ratio: 1.38, first rate 0.6666666666666666
Bonus ratio: 1.4, first 

In [44]:
new_preds = []
for pps in mod_pred_probs:
    new_preds.append([mid for mid, prob in pps])

In [45]:
pred_strs = [
    ' '.join([str(mid) for mid in pred[:25]]) for pred in new_preds
]
submission['MisconceptionId'] = pred_strs
submission

,QuestionId_Answer,MisconceptionId
0,1869_B,55 15 77 24 96 13 90 73 25 46 30 17 34 52 74 2...
1,1869_C,25 15 73 46 96 24 13 30 90 52 77 82 34 74 17 1...
2,1869_D,15 24 46 96 13 25 90 30 73 10 77 65 68 55 74 3...
3,1870_A,59 80 79 46 94 25 73 56 96 24 91 32 3 29 44 6 ...
4,1870_B,59 80 79 46 73 94 96 56 25 24 58 55 82 44 32 3...
5,1870_C,59 80 46 79 94 25 56 24 73 96 3 49 82 64 44 32...
6,1871_A,30 93 46 65 41 79 62 24 25 56 63 13 99 71 64 8...
7,1871_C,30 65 79 94 41 13 93 46 56 24 25 62 63 38 78 6...
8,1871_D,30 56 93 46 94 89 65 62 79 63 24 84 25 49 23 3...


In [46]:
submission.to_csv("submission.csv", index=False)

In [47]:
submission = pd.read_csv("submission.csv")
submission

,QuestionId_Answer,MisconceptionId
0,1869_B,55 15 77 24 96 13 90 73 25 46 30 17 34 52 74 2...
1,1869_C,25 15 73 46 96 24 13 30 90 52 77 82 34 74 17 1...
2,1869_D,15 24 46 96 13 25 90 30 73 10 77 65 68 55 74 3...
3,1870_A,59 80 79 46 94 25 73 56 96 24 91 32 3 29 44 6 ...
4,1870_B,59 80 79 46 73 94 96 56 25 24 58 55 82 44 32 3...
5,1870_C,59 80 46 79 94 25 56 24 73 96 3 49 82 64 44 32...
6,1871_A,30 93 46 65 41 79 62 24 25 56 63 13 99 71 64 8...
7,1871_C,30 65 79 94 41 13 93 46 56 24 25 62 63 38 78 6...
8,1871_D,30 56 93 46 94 89 65 62 79 63 24 84 25 49 23 3...


In [48]:
submission = pd.read_csv("submission_vllm.csv")
submission

,QuestionId_Answer,MisconceptionId
0,1869_B,55 15 77 24 34 52 74 23 64 82 60 96 84 13 90 1...
1,1869_C,15 52 77 82 25 34 74 73 31 68 23 29 46 96 32 6...
2,1869_D,15 77 68 55 24 74 31 64 14 95 46 60 96 13 25 4...
3,1870_A,59 80 91 79 32 3 29 46 94 25 44 6 27 4 55 49 6...
4,1870_B,59 80 79 58 55 46 82 44 32 3 6 64 91 74 29 73 ...
5,1870_C,59 80 46 3 79 49 82 64 44 32 91 74 34 4 89 27 ...
6,1871_A,30 99 93 71 46 65 64 41 86 23 34 16 98 38 2 79...
7,1871_C,30 65 79 38 94 78 64 71 41 98 13 49 14 93 89 7...
8,1871_D,30 89 84 49 23 56 38 93 52 46 94 64 27 99 77 8...
